In [1]:
%load_ext autoreload
%autoreload 2
import gc
import numpy as np
import pandas as pd
import datetime
import glob ,sys , re
from multiprocessing import Pool
import multiprocessing
from categorical_encoding import select_category_value_agg, select_level_agg, select_category_value_agg, cnt_encoding
from mv_wg_avg import exp_weight_avg
from target_encoding import target_encoding

sys.path.append('../../../github/module/')
from preprocessing import set_validation, split_dataset, get_dummies, factorize_categoricals, impute_avg
from convinience_function import get_categorical_features, get_numeric_features, row_number, pararell_process
from logger import logger_func
from feature_engineering import base_aggregation, diff_feature, division_feature
from make_file import make_npy, make_feature_set
from tqdm import tqdm

logger = logger_func()
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 101)

unique_id = 'SK_ID_CURR'
b_id = 'SK_ID_BUREAU'
p_id = 'SK_ID_PREV'
target = 'TARGET'
ignore_features = [unique_id, target, 'valid_no', 'is_train', 'is_test', 'SK_ID_PREV', 'SK_ID_BUREAU']

2018-07-25 22:10:27,398 logger 19 [INFO]    [logger_func] start 
2018-07-25 22:10:27,431 logger 19 [INFO]    [logger_func] start 
2018-07-25 22:10:27,431 logger 19 [INFO]    [logger_func] start 


In [2]:
# ' app '
base = pd.read_csv('../data/base.csv')[unique_id].to_frame()
# app = pd.read_csv('../data/application_train_test.csv')

# ' NULL埋め '
# app['DAYS_EMPLOYED'].fillna(-240, inplace=True)
# app['OCCUPATION_TYPE'].fillna('XNA', inplace=True)

# flag = {'N':0, 'Y':1}
# app['FLAG_OWN_CAR'] = app['FLAG_OWN_CAR'].map(flag)
# app['FLAG_OWN_REALTY'] = app['FLAG_OWN_REALTY'].map(flag)

' bureau '
# bureau = pd.read_csv('../data/bureau_after.csv')
bureau = pd.read_csv('../data/bureau_cleansing.csv')
bureau.sort_values(by=[unique_id, 'SK_ID_BUREAU', 'DAYS_CREDIT'], ascending=False, inplace=True)
bureau = row_number(bureau, unique_id)

base_bureau = bureau[['SK_ID_CURR', 'SK_ID_BUREAU']].drop_duplicates()

' bureau_balanceで遅延したことのあるIDのみJOINする '
balance = pd.read_csv('../data/rawdata/bureau_balance.csv')
balance['STATUS'] =  balance['STATUS'].map(lambda x: '-1' if x=='C' or x=='X' else x).astype('int')
b_status = balance.groupby(b_id)['STATUS'].max()

b_status = b_status.reset_index().query('STATUS>=0')
bureau = bureau.merge(b_status, on=b_id, how='left')

bureau['STATUS'] = bureau['STATUS'] + 1
bureau['DAYS_ENDDATE'] = bureau.apply(lambda x: x['DAYS_ENDDATE_FACT'] if x['DAYS_ENDDATE_FACT'] < 99999 else x['DAYS_CREDIT_ENDDATE'], axis=1)

' ENDDATE_FACT '
# bureau['CREDIT_TERM_FACT'] = bureau['DAYS_ENDDATE_FACT'] - bureau['DAYS_CREDIT']
# bureau['CREDIT_TERM_FACT'] = bureau['CREDIT_TERM_FACT'].map(lambda x:0 if x<0 else x)
# bureau['LOAN_MONTHLY_FACT'] = bureau['AMT_CREDIT_SUM'] / bureau['CREDIT_TERM_FACT']

' ENDDATE '
bureau['CREDIT_TERM'] = bureau['DAYS_ENDDATE'] - bureau['DAYS_CREDIT']
bureau['CREDIT_TERM'] = bureau['CREDIT_TERM'].map(lambda x:0 if x<0 else x)
bureau['LOAN_MONTHLY'] = bureau['AMT_CREDIT_SUM'] / bureau['CREDIT_TERM']

' AMT '
bureau = diff_feature(df=bureau, first='AMT_CREDIT_SUM', second='AMT_CREDIT_SUM_DEBT')
bureau = diff_feature(df=bureau, first='AMT_ANNUITY', second='LOAN_MONTHLY')

In [5]:
path = '../features/5_engineering/*.npy'
fe = make_feature_set(base, path)
df = fe

'positive率とサンプル数によってラベリングする'

' SUITE'
df['a_NAME_TYPE_SUITE'] = df['a_NAME_TYPE_SUITE'].map({
    'Children' : 'middle_row_risk'
    ,'Family' : 'middle_row_risk'
    ,'Unaccompanied' : 'middle_risk'
    ,'Groupofpeople' : 'hight_risk'
    ,'Other_A' : 'high_risk'
    ,'Other_B' : 'high_risk'
    ,'Spouse,partner' : 'row_risk'
})

' INCOME_TYPE'
df['a_NAME_INCｊOME_TYPE'] = df['a_NAME_INCOME_TYPE'].map(lambda x:'Pensioner' if x.count('Business') or x.count('Student') else 'Working' if x.count('Unemployed') or x.count('Maternity') else x)

' EDUCATION_TYPE'
df['a_NAME_EDUCATION_TYPE'] = df['a_NAME_EDUCATION_TYPE'].map(lambda x:'Secondary / secondary special' if x.count('Lower secondary') else 'Higher education' if x.count('Academic') else x)

' EDUCATION_TYPE'
df['a_NAME_FAMILY_STATUS'] = df['a_NAME_FAMILY_STATUS'].map(lambda x:'Married' if x.count('Widow') or x.count('Unknown') else 'Civil marriage' if x.count('Single') or x.count('Separated') else x)

' ORGANIZATION '
tmp = df.query('TARGET != -1')
category = 'a_ORGANIZATION_TYPE'
category = 'a_OCCUPATION_TYPE'
target_avg = tmp.groupby(category)['TARGET'].mean().reset_index()
# print(target_avg)
# sys.exit()

' positive率でカテゴリをビンにする '
bins = 10
bin_list = ['TARGET']
for col in bin_list:
    target_avg[target] = pd.qcut(x=target_avg[col], q=bins, duplicates='drop')
    target_avg.set_index(category, inplace=True)
    target_avg = target_avg.iloc[:,0]
df[category] = df[category].map(target_avg)

' positive率が低い順に番号をつける '
bin_sort = df[category].drop_duplicates().sort_values().to_frame()
bin_sort['index'] = np.arange(len(bin_sort))+1
bin_sort = bin_sort.set_index(category).iloc[:,0]
df[category] = df[category].map(bin_sort)

df.head()
fe = df.copy()

In [6]:
df = bureau

' CREDIT_TYPE'
category='CREDIT_TYPE'
df[category] = df[category].map(lambda x:
                                'high_CREDIT_TYPE' if x.count('Another') or x.count('card') or x.count('Micro') or x.count('equipment') or x.count('replenish')
                                else 'middle_CREDIT_TYPE' if x.count('Consumer') or x.count('Unknown')
                                else 'row_CREDIT_TYPE' if x.count('Car') or x.count('Mortgage') or x.count('develop') or x.count('earmarked') or x.count('Real') or x.count('margin') or x.count('lending') or x.count('Inter') or x.count('Mobile')
                               else 'other'
                               )

' 可視化 '
val_cnt = df[category].value_counts().to_frame()
df = df.query('TARGET>=0')
positive = df.groupby(category)[target].mean().to_frame()
val_cnt.join(positive)

,CREDIT_TYPE,TARGET
middle_CREDIT_TYPE,572711,0.075173
high_CREDIT_TYPE,190505,0.092129
row_CREDIT_TYPE,21951,0.055469


In [5]:
# bureau.query('row_no==1')[[unique_id, b_id]].drop_duplicates().shape
bureau = bureau.merge(df, on=[unique_id, target], how='inner')
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,TARGET,over_flg,progress_flg,row_no,STATUS,DAYS_ENDDATE,CREDIT_TERM,LOAN_MONTHLY,AMT_CREDIT_SUM_diff_AMT_CREDIT_SUM_DEBT@,LOAN_MONTHLY_diff_AMT_ANNUITY@
0,456253,6230038,Active,currency 1,-23.0,NaN,37.0,NaN,NaN,NaN,2250000.0,1529545.5,0.0,NaN,middle_CREDIT_TYPE,0.0,NaN,0.0,0,1,1,1.0,37.0,60.0,37500.000000,720454.5,NaN
1,456253,6098499,Closed,currency 1,-30.0,NaN,-6.0,-26.0,NaN,NaN,675000.0,0.0,0.0,NaN,middle_CREDIT_TYPE,-23.0,58369.5,0.0,0,0,2,1.0,-26.0,4.0,168750.000000,675000.0,110380.500000
2,456253,6098498,Closed,currency 1,-30.0,NaN,-6.0,-26.0,NaN,NaN,675000.0,0.0,0.0,NaN,middle_CREDIT_TYPE,-4.0,58369.5,0.0,0,0,3,1.0,-26.0,4.0,168750.000000,675000.0,110380.500000
3,456253,5113039,Active,currency 1,-30.0,NaN,12.0,NaN,NaN,NaN,360000.0,171036.0,0.0,NaN,high_CREDIT_TYPE,-5.0,58369.5,0.0,0,1,4,1.0,12.0,42.0,8571.428571,188964.0,-49798.071429
4,456249,5371702,Closed,currency 1,-41.0,NaN,-30.0,-30.0,NaN,NaN,540000.0,NaN,NaN,NaN,middle_CREDIT_TYPE,-30.0,NaN,0.0,0,0,1,NaN,-30.0,11.0,49090.909091,NaN,NaN


In [7]:
# num_list = get_numeric_features(data=bureau, ignore=ignore_features)
num_list = ['DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE', 'DAYS_CREDIT_UPDATE', 'AMT_ANNUITY', 'STATUS', 'DAYS_ENDDATE', 'CREDIT_TERM', 'LOAN_MONTHLY', 'AMT_CREDIT_SUM_diff_AMT_CREDIT_SUM_DEBT@', 'LOAN_MONTHLY_diff_AMT_ANNUITY@']

# ['AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_diff_AMT_CREDIT_SUM_DEBT@', 'LOAN_MONTHLY_diff_AMT_ANNUITY@', 'DAYS_CREDIT', 'DAYS_ENDDATE', 'CREDIT_DAY_OVERDUE']
print(num_list)

['DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE', 'DAYS_CREDIT_UPDATE', 'AMT_ANNUITY', 'STATUS', 'DAYS_ENDDATE', 'CREDIT_TERM', 'LOAN_MONTHLY', 'AMT_CREDIT_SUM_diff_AMT_CREDIT_SUM_DEBT@', 'LOAN_MONTHLY_diff_AMT_ANNUITY@']


In [7]:
df = bureau
category = 'NAME_EDUCATION_TYPE'
category = 'NAME_INCOME_TYPE'
category = 'NAME_TYPE_SUITE'
category = 'ORGANIZATION_TYPE'
category = 'OCCUPATION_TYPE'
category = 'CREDIT_ACTIVE'
category = 'CREDIT_TYPE'
val_cnt = df[category].value_counts().to_frame()
df = df.query('TARGET>=0')
positive = df.groupby(category)[target].mean().to_frame()
val_cnt.join(positive)

,CREDIT_TYPE,TARGET
middle_CREDIT_TYPE,572711,0.075173
high_CREDIT_TYPE,190505,0.092129
row_CREDIT_TYPE,21951,0.055469


In [8]:
bureau.sort_values(by=[unique_id, 'SK_ID_BUREAU', 'DAYS_CREDIT'], ascending=False, inplace=True)
bureau = row_number(bureau, [unique_id, 'CREDIT_TYPE'])
# bureau.head(40)

In [ ]:
def bureau_extract_agg(data, bins=0, val='', status='', term='', freq=0):

    if freq>0:
        prefix = f'b_{val}_{status}_latest{freq}_'
    elif freq==0:
        prefix = f'b_{val}_{status}_{term}_'
    
    ' カテゴリー区分 '
    if val.count('high'):
        data = data.query(f"CREDIT_TYPE=='{val}'")
    elif val.count('middle'):
        data = data.query(f"CREDIT_TYPE=='{val}'")
    elif val.count('row'):
        data = data.query(f"CREDIT_TYPE=='{val}'")
    
    ' ステータス区分 '
    if status.count('Act'):
        data = data.query(f"CREDIT_ACTIVE=='{status}'")
    elif status.count('Clo'):
        data = data.query(f"CREDIT_ACTIVE=='{status}'")
    elif status.count('og_a'):
        data = data.query(f"DAYS_ENDDATE!=DAYS_ENDDATE or DAYS_ENDDATE>0")
    elif status.count('og_c'):
        data = data.query(f"DAYS_ENDDATE<=0")
        
    ' 期間 '
    if term.count('3mon'):
        data = data.query(f"DAYS_CREDIT>=-3")
    elif term.count('6mon'):
        data = data.query(f"DAYS_CREDIT>=-6")
    elif term.count('1year'):
        data = data.query(f"DAYS_CREDIT>=-12")
    elif term.count('2year'):
        data = data.query(f"DAYS_CREDIT>=-24")
    elif term.count('3year'):
        data = data.query(f"DAYS_CREDIT>=-36")
        
    if freq>0:
        data = data.query(f'row_no<={freq}')
    
    ' dataチェック '
#     print(data.shape)
#     print('*** SK_ID_CURR ***')
#     print(data[unique_id].drop_duplicates().shape)
    ' ユニークIDが1万に満たない場合は特徴量としない '
    if data[unique_id].drop_duplicates().count()<10000:
        print(f'less than 10000 unique_id. combi: {bins}_{val}_{status}_{term}_{freq}')
        return
    
    print(f'combi: {bins}_{val}_{status}_{term}_{freq}')
    print(f'unique_id count: {data[unique_id].drop_duplicates().count()}')
#     return
    # sys.exit()
    
    ext = data.copy()
    del data
    gc.collect()
    
    
    days_list = [col for col in ext.columns if col.count('DAYS')]
    amt_list = [col for col in ext.columns if col.count('AMT_')]
    # print(days_list)
    # print(amt_list)
    
    amt = ext[[unique_id] + amt_list].groupby(unique_id).agg('sum')
    days = ext[[unique_id] + days_list].groupby(unique_id).agg('max')
    result = amt.join(days)
    
    for col in result.columns:
        if col in ignore_features:
            continue
        elif col.count('AMT_'):
            result.rename(columns={col:f'{col}_sum@'}, inplace=True)
        elif col.count('DAYS_'):
            result.rename(columns={col:f'{col}_max@'}, inplace=True)
            
    ' ここからターゲットエンコーディング '
    method_list = ['mean', 'std']
    # method_list = ['sum', 'mean', 'std', 'max', 'min']
    
    val_col = 'valid_no_4'
    bin_list  = [col for col in result.columns if col.count('@')]
    result = result.merge(fe, on=[unique_id], how='inner')
    
    cat_list_1 = []
    # cat_list_1 = [
    #     'a_OCCUPATION_TYPE'
    #     ,'a_ORGANIZATION_TYPE'
    #     ,'a_NAME_EDUCATION_TYPE'
    #     ,'a_NAME_INCOME_TYPE'
    #     ,'a_CODE_GENDER'
    #     ,'a_NAME_TYPE_SUITE'
    #     ,'a_NAME_FAMILY_STATUS'
    #     ,'a_REGION_RATING_CLIENT_W_CITY'
    #     ,'a_HOUSE_HOLD_CODE@'
    #     ,'a_FLAG_WORK_PHONE'
    # ]
    
    cat_list_2 = [
        'a_OCCUPATION_TYPE'
        ,'a_ORGANIZATION_TYPE'
        ,'a_NAME_EDUCATION_TYPE'
        ,'a_NAME_INCOME_TYPE'
        ,'a_CODE_GENDER'
        ,'a_NAME_FAMILY_STATUS'
        ,'a_NAME_TYPE_SUITE'
        ,'a_REGION_RATING_CLIENT_W_CITY'
        ,'a_HOUSE_HOLD_CODE@'
        ,'a_FLAG_WORK_PHONE'
    ]
    
    ' binをカテゴリとする場合はこちら '
    for col in bin_list:
        print(col)
        ' binより値の種類が少ない時は計算しない '
        if len(result[col].drop_duplicates())<bins:
            cat_list_1.append(col)
        else:
            result[f'bin{bins}_{col}'] = pd.qcut(x=result[col], q=bins, duplicates='drop')
            cat_list_1.append(f'bin{bins}_{col}')
        
    #     ' 可視化チェック '
    #     print(result[f'bin{bins}_{col}'].value_counts())
    #     result.drop(f'bin{bins}_{col}', axis=1, inplace=True)
    #     sys.exit()
        
    
    categorical = []
    for cat1 in cat_list_1:
        for cat2 in cat_list_2:
            if cat1==cat2:continue
            categorical.append([cat1, cat2])
    
    ' エンコーディングするカテゴリカラムのリストを作る '
    #  categorical = get_categorical_features(result, ignore_features)
    
    ' カテゴリの組み合わせをエンコーディングする場合はこちら '
    #  cat_combi = list(combinations(categorical, 2))
    #  categorical = cat_combi
    
    select_list = []
    #  categorical = [col for col in result.columns if col.count('cluster')]
    
    for cat in categorical:
        target_encoding(base=base, data=result, unique_id=unique_id, level=cat, method_list=method_list, prefix=prefix, select_list=select_list, test=1, impute=1208, val_col=val_col)
    

'集計前の絞り込み条件'
'SK_ID_CURRが1万件以上のデータに限る'
data = bureau.copy()
bins = 30
bins_list = [20, 10, 5]

val = ''
# val = 'high_CREDIT_TYPE'
# val = 'middle_CREDIT_TYPE'
# val = 'row_CREDIT_TYPE'
val_list = ['', 'high_CREDIT_TYPE' ,'middle_CREDIT_TYPE' ,'row_CREDIT_TYPE']
# val_list = ['']

status = ''
# status = 'Active'
# status = 'Closed'
# status = 'og_a'
# status = 'og_c'
status_list = ['', 'Active' ,'Closed' ,'og_a' ,'og_c']

term = ''
# term = '3month'
# term = '6month'
# term = '1year'
# term = '2year'
# term = '3year'
term_list = ['3month' ,'6month' ,'1year' ,'2year' ,'3year']

freq = 0
# freq = 3
# freq = 5
# freq = 10
freq_list = [3, 5, 10]

number = 0
# for bins in bins_list:
for bins in [20, 10, 5]:
#     for val in val_list:
    for val in ['middle_CREDIT_TYPE', 'row_CREDIT_TYPE']:
        if val.count('middle') and bins==20:continue
        for status in status_list:
#             for term in term_list:
            for freq in freq_list:
                number += 1
                if number<10:
                    continue
                bureau_extract_agg(data=data, bins=bins, val=val, status=status, freq=freq)
sys.exit()            
for bins in bins_list:
    for val in val_list:
        for status in status_list:
            for term in term_list:
                bureau_extract_agg(data=data, bins=bins, val=val, status=status, term=term)

KeyError: 'a_OCCUPATION_TYPE'